<a href="https://colab.research.google.com/github/vladsavelyev/deeplearning/blob/master/pretgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
DRIVE_PATH = Path("/content/drive/MyDrive")
!pip install torch transformers datasets tokenizers evaluate wandb

In [ ]:
from pathlib import Path
import numpy as np

import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, TrainerCallback

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2")
model.config.vocab_size

50264

In [ ]:
len(tokenizer.vocab)

50258

In [ ]:
class TransformerDataset(Dataset):
    def __init__(self, token_ids: np.memmap, n_ctx: int):
        self.token_ids = token_ids
        self.n_ctx = n_ctx

    def __getitem__(self, idx):
        t = torch.LongTensor(self.token_ids[idx:idx + self.n_ctx])
        return {"input_ids": t, "labels": t}

    def __len__(self):
        return len(self.token_ids) - self.n_ctx + 1

    @staticmethod
    def load(path, tokenizer, n_ctx: int, max_n_examples: int = None) -> 'TransformerDataset':
        save_path = path.with_suffix(".token_ids.pt")
        if False and save_path.exists():
            print(f"Loading dataset from {save_path}")
            ids = torch.load(str(save_path))
        else:
            with open(path, "r") as f:
                text = f.read()
                print(f"Characters in text: {len(text):,}")
            ids = tokenizer(text, return_tensors="pt")['input_ids'].squeeze().long()
            if max_n_examples:
                max_tokens = max_n_examples + n_ctx - 1
                print(f"Taking first {max_tokens} tokens to make it {max_n_examples} examples")
                ids = ids[:max_tokens]
            eos = torch.tensor([tokenizer.eos_token_id]).long()
            ids = torch.concat((ids, eos))
            torch.save(ids, save_path)
        print(f"Dataset shape: {ids.shape}")
        return TransformerDataset(ids, n_ctx)


test_text_path = DRIVE_PATH / "AI" / "datasets" / "murakami" / "murakami_test.txt"
train_text_path = DRIVE_PATH / "AI" / "datasets" / "murakami" / "murakami_train.txt"
test_set = TransformerDataset.load(test_text_path, tokenizer, model.config.n_ctx, max_n_examples=100)
train_set = TransformerDataset.load(train_text_path, tokenizer, model.config.n_ctx)

Characters in text: 27,685
Taking first 2147 tokens to make it 100 examples
Dataset shape: torch.Size([2148])
Characters in text: 10,127,380
Dataset shape: torch.Size([2503610])


In [ ]:
test_set.token_ids.min(), test_set.token_ids.max()

(tensor(5), tensor(50257))

In [ ]:
train_set[0]['input_ids'].shape

torch.Size([2048])

In [ ]:
print(f'{sum([p.numel() for p in model.parameters()]):,}')

125,231,616


In [ ]:
model.config.n_ctx

2048

In [ ]:
b = train_set[1000]
res = model(input_ids=b['input_ids'].view(1, -1), labels=b['labels'].view(1, -1))
print(res['loss'])

t = tokenizer("Определенно, это случилось с нами не в первый раз")['input_ids']
t = torch.LongTensor(t).view(1, -1)
res = model(input_ids=t, labels=t)
print(res['loss'])

tensor(4.5174, grad_fn=<NllLossBackward0>)
tensor(4.4814, grad_fn=<NllLossBackward0>)


In [ ]:
from transformers.trainer_utils import get_last_checkpoint
save_dir = DRIVE_PATH / "AI" / "pretgpt" / "murakami_rugpt3small"
save_dir.mkdir(exist_ok=True)
if (last_checkpoint_dir := get_last_checkpoint(str(save_dir))):
    last_checkpoint_dir = Path(last_checkpoint_dir)
    print(last_checkpoint_dir)
    print('  '.join(t.name for t in last_checkpoint_dir.iterdir()))

/content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-11000
config.json  generation_config.json  pytorch_model.bin  training_args.bin  optimizer.pt  scheduler.pt  trainer_state.json  rng_state.pth


In [ ]:
!ls /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/

checkpoint-11000


In [ ]:
# from transformers.utils import WEIGHTS_NAME
# state_dict = torch.load(str(last_checkpoint_dir / WEIGHTS_NAME), map_location="cpu")
# model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
def sample(num_seqs=10, max_length=10):
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    for i, seq in enumerate(model.generate(
        max_length=max_length,
        top_p=0.95,
        num_return_sequences=num_seqs,
        do_sample=True, 
        top_k=50,
        pad_token_id=0,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )):
        print(i + 1, tokenizer.decode(seq))

sample()

1 <|endoftext|>, или что-то вроде этого.

2 <|endoftext|>.  И вот что я нашел. 

3 <|endoftext|>.

— Это что, шутка
4 <|endoftext|> и без того некрасивую фигуру,
5 <|endoftext|> - &quot;природа в человеке&
6 <|endoftext|>.  Поэтому, когда вы выбираете одежду
7 <|endoftext|> на работу не берут





8 <|endoftext|> к этой теме.  В качестве примера возьмем
9 <|endoftext|>, который для меня был загадкой. Я не
10 <|endoftext|> по адресу: город Москва, ул. Большая


In [ ]:
import os
os.environ['WANDB_API_KEY'] = '270e81630bd1fd3c78a355d1711966c75ce75bcc'
os.environ["WANDB_NOTEBOOK_NAME"] = "pretgpt"
from importlib import reload
import wandb
reload(wandb)

<module 'wandb' from '/usr/local/lib/python3.8/dist-packages/wandb/__init__.py'>

In [ ]:
from transformers.trainer_utils import get_last_checkpoint
save_dir = DRIVE_PATH / "AI" / "pretgpt" / "murakami_rugpt3small"
save_dir.mkdir(exist_ok=True)
if (last_checkpoint_dir := get_last_checkpoint(str(save_dir))):
    last_checkpoint_dir = Path(last_checkpoint_dir)
    print(last_checkpoint_dir)
    print('  '.join(t.name for t in last_checkpoint_dir.iterdir()))

class MyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if metrics:
            print(f'Eval loss so far: {metrics["eval_loss"]:.4f}')
        if state.best_metric:
            print(f"Best loss so far: {state.best_metric:.4f}")
        sample()

trainer = Trainer(
    model=model,
    train_dataset=train_set,
    eval_dataset=test_set,
    callbacks=[MyCallback],
    args=TrainingArguments(
        output_dir=str(save_dir),
        report_to=['wandb'],
        overwrite_output_dir=True,
        evaluation_strategy="steps",
        eval_steps=1000,
        save_steps=1000,
        save_total_limit=2,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        ignore_data_skip=True,
    ),
)
trainer.train(resume_from_checkpoint=last_checkpoint_dir)

PyTorch: setting up devices
Loading model from /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-1000.


/content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-1000
config.json  generation_config.json  pytorch_model.bin  training_args.bin  optimizer.pt  scheduler.pt  trainer_state.json  rng_state.pth


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2501563
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 3752346
  Number of trainable parameters = 125231616
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 1000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
2000,2.379800,3.487939
3000,2.406300,3.641360
4000,2.282300,3.718564
5000,2.146300,3.831849
6000,2.009200,3.990011
7000,1.903500,4.049981
8000,1.834900,4.235762
9000,1.718300,4.300036
10000,1.627800,4.345162
11000,1.551100,4.461396


***** Running Evaluation *****
  Num examples = 101
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



...

Eval loss so far: 4.3452
1 <|endoftext|> у них действительно есть. Вы понимаете, о
2 <|endoftext|>. А может, и нет. Как бы
3 <|endoftext|>.
— Если честно, я все это
4 <|endoftext|>. И это — одна из проблем, которые
5 <|endoftext|>. В любом случае, на свете, возможно
6 <|endoftext|> назад.
– Так или иначе – сейчас
7 <|endoftext|>:
– В общем, на самом деле
8 <|endoftext|>, и она по очереди снимает очки.

9 <|endoftext|>. Все, что касалось моей работы, решено
10 <|endoftext|>. В комнате повисла такая тишь, будто


Saving model checkpoint to /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-10000
Configuration saved in /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-10000/config.json
Configuration saved in /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-10000/generation_config.json
Model weights saved in /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-10000/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-8000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 101
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



Eval loss so far: 4.4614
1 <|endoftext|> у окна.
У меня есть сестра.
2 <|endoftext|>. А может, и нет. Как бы
3 <|endoftext|>.
— Если честно, — сказал я
4 <|endoftext|>. И это — важнейшая часть его жизни
5 <|endoftext|>. В любом случае, на обратном пути,
6 <|endoftext|> назад.
– Так, может, и
7 <|endoftext|>, но ничего примечательного она от себя не
8 <|endoftext|>, и она по очереди снимает очки.

9 <|endoftext|>. Все, что касалось моей работы, решено
10 <|endoftext|>.
У меня в голове еще несколько раз


Saving model checkpoint to /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-11000
Configuration saved in /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-11000/config.json
Configuration saved in /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-11000/generation_config.json
Model weights saved in /content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-11000/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-9000] due to args.save_total_limit


KeyboardInterrupt: ignored

In [ ]:
sample(num_seqs=1, max_length=1000)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



1 <|endoftext|>-флейте у моей двери была дверь в ванную. В коридоре я стоял на шаг-парадном и разглядывал цветочки.
– Это, похоже, тюльпан, – сказал я.
– Бальзаматы? – Она обернулась и взглянула на меня так, словно видела перед собой какую-то редкую зверушку.
– Угу.
– Ты что, на тюленье окно похож?
– Конечно.
– А это что, действительно тюльпан?
– Конечно.
– А откуда он растет?
Я пошарил в карманах куртки, нащупал коробку с тюльпаном и тюльпаном и снова дернув за ручку, открыл дверь. Пока я искал тюльпан и вешалку, дождь все лил и лил – настолько обильно, что в ванной заискрился шербет. «Тюльпановы» у меня дома – коллекция еще с тех времен, когда мне семья выдраного корчилола глазки.
– 
В ванной мы оба долго ждали, пока он выйдет, – чтобы обнаружить, что кто-то снова там появится.
В конце концов дверь приоткрылась и стряхнула с себя капли. На миг в ванной посветлели, поблекли и померкли все краски. Сдохнув, дверь подалась на петлях, и на меня в окутавшую нас влагу вдруг пролился томител

In [ ]:
from pathlib import Path

from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

from transformers.trainer_utils import get_last_checkpoint
save_dir = DRIVE_PATH / "AI" / "pretgpt" / "murakami_rugpt3small"
save_dir.mkdir(exist_ok=True)
if last_checkpoint_dir := get_last_checkpoint(str(save_dir)):
    last_checkpoint_dir = Path(last_checkpoint_dir)
    print(last_checkpoint_dir)
    print('  '.join(t.name for t in last_checkpoint_dir.iterdir()))

from transformers.utils import WEIGHTS_NAME
import torch
state_dict = torch.load(str(last_checkpoint_dir / WEIGHTS_NAME), map_location="cpu")
model.load_state_dict(state_dict)    

def generate(**kwargs):
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    seq = model.generate(
        max_length=500,
        num_return_sequences=1,
        pad_token_id=0,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        **kwargs,
    )[0]
    print(tokenizer.decode(seq))
    print()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/content/drive/MyDrive/AI/pretgpt/murakami_rugpt3small/checkpoint-11000
config.json  generation_config.json  pytorch_model.bin  training_args.bin  optimizer.pt  scheduler.pt  trainer_state.json  rng_state.pth


In [ ]:
print("My search:")
generate(
    do_sample=True, 
    top_p=0.95,
    top_k=50,
)

My search:
<|endoftext|>, а волосы были пострижены коротко и совсем коротко. А сам он, долговязый, как толстокожий, почти не подавал виду, но стоило ему приподнять голову, как на его лице появлялась какая-то удивительная улыбка. Всякий раз, когда я его замечал, в ответ эти губы складывались в неопределенную улыбку, которая мне так нравился.
– Ты здесь много времени проводишь? – поинтересовался я.
– А ты разве не в Токио?
– В Токио я был последний раз, – ответил я.
– И где же?
– Не знаю. Но не так далеко, к западу от Токио.
Я вспомнил, как мы расстались прошлым летом, как ходили вдвоем в поход по горам Яманотэ. Вспомнил и тот жаркий день – когда мы расстались с Кидзуки.
– А ты почему развелся? – спросил я.
В ответ – молчание. Казалось, она о чем-то задумалась.
– А я здесь пять лет жил. Работал на складе.
– Мой холодильник не работает.
– Извини. Я должен идти. А ты?
Я покачал головой.
– Знаешь… – сказала она. – Когда все успокоится, я уйду из этого дома. Останусь здесь, здесь, а не здесь

In [ ]:
print("Greedy search:")
generate()

Greedy search:
<|endoftext|>.
– А что, если я попробую с ним поговорить?
– Конечно, – ответил я. – Только учти, что я не смогу говорить с ним, пока ты не встретишься с ним.
– Хорошо, – сказала она. – Но учтите, что я не смогу поговорить с ним, пока ты не встретишься с ним.
– Хорошо, – сказал я.
Она положила руку мне на плечо.
– Я не знаю, как это сказать… но мне очень страшно.
– Не страшно?
– Да.
– Мне очень страшно, – повторила она. – Не представляю, что это значит.
– Я не знаю, что это значит, но… мне очень страшно.
– Ну, не знаю… – сказала она. – Но это значит, что я не смогу ему помочь.
– Каким образом?
– Не знаю. Но я уверена, что он – это он.
– И он – твой друг?
– Да.
– Значит, он – твой друг?
– Да.
– Значит, я смогу с ним поговорить?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Ду

In [ ]:
print("Nucleus sampling:")
generate(
    do_sample=True,
    top_p=0.05,
    top_k=0,
)

Nucleus sampling:
<|endoftext|>.
– А что, если я попробую с ним поговорить?
– Конечно, – ответил я. – Только учти, что я не смогу говорить с ним, пока ты не встретишься с ним.
– Хорошо, – сказала она. – Но учтите, что я не смогу поговорить с ним, пока ты не встретишься с ним.
– Хорошо, – сказал я.
Она положила руку мне на плечо.
– Я не знаю, как это сказать… но мне очень страшно.
– Не страшно?
– Да.
– Мне очень страшно, – повторила она. – Не представляю, что это значит.
– Я не знаю, что это значит, но… мне очень страшно.
– Ну, не знаю… – сказала она. – Но это значит, что я не смогу ему помочь.
– Каким образом?
– Не знаю. Но я уверена, что он – это он.
– И он – твой друг?
– Да.
– Значит, он – твой друг?
– Да.
– Значит, я смогу с ним поговорить?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
– Думаю, да.
– Значит, я смогу поговорить с ним?
–

In [ ]:
print("Contrastive search:")
generate(
    penalty_alpha=0.6, 
    top_k=4,
)

Contrastive search:
<|endoftext|> в том, что это место обладает мощной энергией.
– Мощной?
– Да, излучающей волны высокой четкости. Она излучает не частоту, а импульс длительностью порядка десяти секунд. У меня в памяти это называется «параллельный мир». Но далеко не все, что я знаю, связано с этим местом. Есть у него какая-то связь со мной, а есть ли она у тебя, не знаю. Я думаю, это взаимосвязно.
– Что-то вроде родника? – предположил я.
– Нет, скорее всего, нет. Источник находится где-то на дне ущелья, окруженном высокими каменными стенами. И вода в нем – как из бочки, которую замуровали в склеп за кумирней. В мире, где отсутствуют звуки, которые можно слышать, и нет времени, чтобы насытиться этой водой. Она доступна только в подземном мире. Этот мир называется «Пространство А» и охраняется очень строгой дисциплиной. По религиозным соображениям туда не вход воспрещен. Однако верующие, католики и верующие-католики категорически против того, чтобы туда стекалась вода из других мест. Он

In [ ]:
print("Beam search:")
generate(
    do_sample=False,
    num_beams=4,
)

Beam search:
<|endoftext|>.
– Я не знаю, как это получилось, но, похоже, так оно и было, – сказал я.
– Я тоже так думаю, – сказала она.
– Я тоже, – сказал я.
– И все-таки?
– Все-таки, – сказал я.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней. – Все-таки, все-таки…
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки? – повторила она за мной.
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я за ней.
– Все-таки?
– Все-таки, – повторил я 

In [ ]:
print("Beam multinomial search:")
generate(
    do_sample=True,
    num_beams=4,
)


Beam multinomial search:
<|endoftext|>.
– Ну, что? Поехали дальше?
– Да-да, конечно, – сказал я.
– Ну, что? Поехали дальше?
– Да-да, конечно, – сказал я.
– Ну, счастливо тебе, – сказала она.
– Пока, – ответил я.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – повторил я.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – повторил я.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – повторил я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – повторил я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказала она.
– Пока, – сказал я.
– Пока, – сказала она.
– Пока, – сказала она.
Пока, – сказала она.
Пока, – сказала она.
Пока, – сказала она.
Пока, – сказал я.
Пока, – сказала она.
Пока, – сказала она.
Пока, – сказала она.
По